# Ranking for property

In [56]:
import sys
sys.path.append("../repository/")
import database as db
import polars as pl
from math import floor

In [57]:
conn_wgi = db.DatabaseConnection("world-governance-index").connection
conn_ti = db.DatabaseConnection("transformation-index").connection
conn_ef = db.DatabaseConnection("economic-freedom").connection
conn_db = db.DatabaseConnection("doing-bussiness").connection

In [58]:
ef_expression = pl.read_database(
    query="""
    SELECT country_id, freedom_of_foreigners_to_visit 
    FROM "freedom" 
    WHERE "year" = 2020 
    ORDER BY country_id
    """,
    connection=conn_ef,
)

ti_expression = pl.read_database(
    query="""
    SELECT country_id, freedom_of_expression 
    FROM "freedom" 
    WHERE "year" = 2020 
    ORDER BY country_id
    """,
    connection=conn_ti,
)

wgi_expression_rank = pl.read_database(
    query="""
    SELECT country_id, rank 
    FROM "VoiceAccountability" 
    WHERE "year" = 2020
    ORDER BY country_id
    """,
    connection=conn_wgi,
)

In [59]:
ef_expression = ef_expression.sort(by="freedom_of_foreigners_to_visit", descending=True)
ef_rank = ef_expression.with_columns(
    ef_rank=ef_expression["freedom_of_foreigners_to_visit"].rank("ordinal", descending=True)
).select(pl.exclude("freedom_of_foreigners_to_visit"))

In [60]:
ti_expression = ti_expression.sort(by="freedom_of_expression", descending=True)
ti_rank = ti_expression.with_columns(
    ti_rank=ti_expression["freedom_of_expression"].rank("ordinal", descending=True)
).select(pl.exclude("freedom_of_expression"))

In [61]:
wgi_rank = wgi_expression_rank.rename({"rank": "wgi_rank"})

In [62]:
df = (
    wgi_rank.join(ef_rank, on="country_id", how="left")
    .join(ti_rank, on="country_id", how="left")
)

df.sort(by=[ "ti_rank", "ef_rank", "wgi_rank"], nulls_last=True)

country_id,wgi_rank,ef_rank,ti_rank
i64,f64,u32,u32
44,79.71,48,1
53,85.99,30,2
69,87.92,12,3
111,66.67,69,4
237,93.72,105,5
10,65.7,38,6
11,48.79,39,7
129,80.68,19,8
225,67.15,null,9
